# Mixed effects modeling of alpha power from EEG data
This tutorial shows how to use either `python` or `R` to run a mixed-effects regression model. We will simulate some rather simple data --- alpha power in each epoch --- with known properties, so we can assess how well the model recovers the structure of the data.  Note that we'll be trying to predict a **scalar** value for each epoch; things get more complicated if you want to use the epoch time series as the thing being predicted.

## Setting up the workspace
We begin by importing the various python packages we'll need, and setting a random seed so our simulated data is reproducible.  We'll also change the default figure size, just so it fills up the notebook width.  Later, we'll import `rpy2` so we can run both `R` and `python` code in the same notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as formula
import mne

rng = np.random.RandomState(12345)
plt.rc('figure', figsize=(16, 12))

If you are running this in a Jupyter Notebook, there is a "magic" command `%load_ext rpy2.ipython` that does all that automatically, but sadly the `mne` documentation is auto-built and doesn't support magic commands.

## Simulating the data
Next we'll simulate alpha power measurements for 12 subjects and 2 conditions ("noun" and "verb"). Each condition will consist of 20 items, with 10 repetitions per item.

In [2]:
subjs = range(12)
conds = ['noun', 'verb']
nouns = ['time', 'year', 'people', 'way', 'day', 'man', 'thing', 'woman',
         'life', 'child', 'world', 'school', 'state', 'family', 'student',
         'group', 'country', 'problem', 'hand', 'part']
verbs = ['be', 'have', 'do', 'say', 'go', 'get', 'make', 'know', 'see', 'say',
         'think', 'come', 'want', 'use', 'find', 'give', 'tell', 'work',
         'call', 'try']
words = nouns + verbs
trials_per_word = 10
trials = range(len(words) * trials_per_word)

Next we'll specify the parameters of the model that we'll be trying to recover later. Since we're modeling "alpha power", these variables determine what the alpha power will be for a given subject/trial combination.  For example, in `cond_intercepts` we're specifying that nouns, on average, will have alpha power that is 7 units greater than verbs, and additionally (in `word_intercepts`) specifying some per-word random deviations from those condition averages.

In [3]:
global_intercept = 100
cond_intercepts = np.repeat([7, 0], [len(nouns), len(verbs)])
word_intercepts = rng.randn(len(words)) * 1.5
# pre-combine condition and word effects into a single offset, 
# and tile it to get the total number of trials
trial_intercepts = np.tile(cond_intercepts + word_intercepts, trials_per_word)

subj_intercepts = rng.randn(len(subjs)) * 4
subj_noise = np.exp(1 + rng.randn(len(subjs)))  # TODO: this is being added as an intercept,
                                                # but should really be a subject-level variance?
eps = rng.randn(len(subjs), len(trials))

We also specified in `subj_intercepts` that each subject will vary in her overall alpha power (e.g., because of differences in alertness, skull thickness, EEG sensor position, etc), and add in some noise. Now we're ready to compute the alpha power values for each epoch:

In [4]:
alpha_powers = (global_intercept + 
                subj_intercepts[:, np.newaxis] + 
                trial_intercepts[np.newaxis, :] +
                subj_noise[:, np.newaxis] * eps)

## Generate signals
Next we generate the actual signals.  For simplicity we're just doing a single channel, since we're going to reduce it back down to scalar alpha-band power anyway.

In [5]:
fs = 1000
epoch_dur = 2.
times = np.linspace(0, epoch_dur, int(epoch_dur * fs))

fourier_freqs = np.fft.rfftfreq(len(times), d=1/fs)
alpha_band_freqs = np.where(np.logical_and(fourier_freqs > 8, 
                                           fourier_freqs < 12))[0]
other_band_freqs = np.where(np.logical_or(fourier_freqs < 7, 
                                          fourier_freqs > 13))[0]
fourier_coefs = np.zeros((len(subjs), len(trials), len(fourier_freqs)))
# for simplicity, we'll just put all the power in a single frequency bin
# near the middle of the alpha band
freq_bin = len(alpha_band_freqs) // 2
fourier_coefs[:, :, alpha_band_freqs[freq_bin]] = np.sqrt(alpha_powers)

# generate the signals
signal = np.fft.irfft(fourier_coefs, axis=-1)

# make sure we didn't screw up:
powr = np.fft.rfft(signal, axis=-1)
powr = (powr * powr).sum(axis=-1)
assert np.allclose(alpha_powers, powr)

# add some energy outside alpha band
coefs = rng.rand(len(subjs) * len(trials) * len(other_band_freqs)
                 ).reshape(len(subjs), len(trials), -1)
fourier_coefs[:, :, other_band_freqs] = coefs
signal_plus_noise = np.fft.irfft(fourier_coefs, axis=-1)
signal_plus_noise[:, :, 0] = 0.  # hack to avoid cosine phase issue

# TODO: signal appears the same in all trials below
# plot a few trials, just to see
fig, axs = plt.subplots(2, 2)
for trial, ax in zip([0, 20, 50, 90], axs.ravel()):
    ax.plot(signal_plus_noise[0, trial, :])
    ax.plot(signal[0, trial, :])

## Generate `Epochs` objects
Now that we have some (fake) data, let's get it into `mne-python` format.  If you're working with real data, you would skip all the synthesis above and start here.

In [6]:
# TODO: create event dict, etc
# info = XXX
# events = np.zeros((len(trials), 3))
# event_id = dict()
# epochs = mne.epochs.EpochsArray(signal_plus_noise, info, events, proj=False)
# TODO: Recover alpha power from noisy signals

## Convert data to long form for modeling
For now, we'll skip all the signal / epoch generation above, and just work with the generated alpha powers (on the assumption that they would have been recovered more or less as-is from the noisy signals).  When the signal/epoch generation code above is completed, we'll swap in `recovered_alpha_powers` for `alpha_powers` in what follows.

In [7]:
variables = dict(alpha_power=alpha_powers.ravel(),
                 subj=np.repeat(subjs, len(trials)),
                 word=np.tile(words, len(subjs) * trials_per_word),
                 cond=np.tile(np.repeat(conds, [len(nouns), len(verbs)]), 
                              len(subjs) * trials_per_word))

dfrm = pd.DataFrame(variables)

## Do the stats

First, we will consider the following **mixed-effects model**:

\begin{equation}
  \log y_{c,j,i} = \beta_0 + \beta_c + \beta_j + \epsilon_{c,j,i}
\end{equation}

Where $\epsilon_{c,j,i} \sim \mathcal{N}(0, \sigma_j)$.

**TODO** add more explanation

In [8]:
md = formula.mixedlm("alpha_power ~ cond", dfrm, groups=dfrm['subj'])
fitmod_py = md.fit() 
print(fitmod_py.summary())

           Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: alpha_power
No. Observations:   4800    Method:             REML       
No. Groups:         12      Scale:              26.1812    
Min. group size:    400     Likelihood:         -14679.4752
Max. group size:    400     Converged:          Yes        
Mean group size:    400.0                                  
-----------------------------------------------------------
              Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
-----------------------------------------------------------
Intercept    105.880    1.216  87.077 0.000 103.497 108.263
cond[T.verb]  -7.709    0.148 -52.193 0.000  -7.999  -7.420
groups RE     17.611    1.479                              



## Run the same model in R
Now we'll use `rpy2` to run the same model using the `lme4` package in `R`.  `rpy2.robjects.r` is the interface to the running `R` instance, and is both callable (e.g., `r('rpois(5, 2)')`) and also acts as a `dict` of objects in `R`'s global environment (e.g., `r['rpois'](5, 2)`).  Aside from that, we need the ability to import `R` packages (`importr`) and a few other objects that allow us to convert a `pandas` DataFrame into an `R` DataFrame.

In [9]:
from rpy2.robjects import r
from rpy2.robjects.packages import importr

# convert pandas dataframe into R dataframe
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri, default_converter, DataFrame
with localconverter(default_converter + pandas2ri.converter) as cv:
    dfrm_r = DataFrame(dfrm)

# load the mixed-modeling package
lme4 = importr('lme4')
fitmod_r = lme4.lmer('alpha_power ~ cond + (1|subj)', data=dfrm_r)

# for some reason, the summary function prints all the data too...
# (that doesn't happen with the "magic" version below). This works too, but
# also print all the data: print(lme4.summary_merMod(fitmod_r)) 
print(r['summary'](fitmod_r))

Linear mixed model fit by REML ['lmerMod']
Formula: alpha_power ~ cond + (1 | subj)
   Data: structure(list(alpha_power = c(99.5465227958328, 95.7130570241655,  
96.7200582555291, 90.539029151465, 98.7583148223078, 96.3797952639812,  
94.1219915728183, 100.871626771234, 100.659725511496, 101.285479950294,  
101.76348397482, 94.2665851022979, 96.3032410717816, 95.4312241424125,  
97.6409241862295, 99.4907172890311, 94.6977156109933, 93.8946612125293,  
102.259724694815, 98.0692449195121, 92.4677836270772, 96.1798352333942,  
92.7561780771036, 91.3153660692029, 89.3432462309628, 93.1994601558047,  
91.5218221252841, 97.1756533782437, 90.2550116963725, 97.9604296790976,  
89.5715484464458, 91.4518266534307, 87.3242471501705, 85.7604563390065,  
90.8425514215158, 91.8126669669946, 84.4577088061296, 89.230953618694,  
88.8793127363421, 88.5661253235318, 95.9589625841874, 98.1833267778681,  
99.7256353837249, 99.3637279624549, 98.9274966360625, 103.90124238439,  
96.2813836947679, 99.2125557

Again, there is a "magic" command that makes this much easier in Jupyter notebooks; just put `%%R` as the first line of a cell, and the rest of the cell will be interpreted as `R` code and sent to `R` through the `rpy2` interface.  To access `python` variables in the notebook environment, you can import them on the first line like this:
```
%%R -i name_of_python_variable
```
To send variables created or modified in the `R` cell back to `python`, use the `-o` flag in the same way:
```
%%R -o name_of_r_variable
```
You can also combine the `-i` and `-o` flags on one line to both import into and export out of a single `R` cell.  Putting all this together, the `R` code in the cell above gets reduced to:

```
%%R -i dfrm -o fitmod_r
library(lme4)  # may want to wrap with suppressWarnings(...)
fitmod_r <- lmer(alpha_power ~ cond + (1|subj), data=dfrm)
summary(fitmod_r)
```